![](https://informatica.ucm.es/themes/ucm16/media/img/logo.png)


## NoSQL

#### Práctica 2

Por favor, lo primero poner los nombres de los integrantes del grupo:


Componente 1:Daniela Nicoleta Boldureanu

Componente 2: Adrián Nieto Moya



## Datos de entrada

Partimos del siguiente fichero json con resultados de las Pruebas Pisa por países en el año 2015

[pisa.json](https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisa.json)

debemos descargar este fichero a nuestro ordenador y luego importarlo con:

    mongoimport  -d educacion -c pisa --drop --file pisa.json

El aspecto del fichero:

    {"_id":1,"pais":"Albania","continente":"Europe","materia":"MAT","nota":413.157}
    {"_id":2,"pais":"Albania","continente":"Europe","materia":"MAT_FE","nota":417.7500295}
    {"_id":3,"pais":"Albania","continente":"Europe","materia":"MAT_MA","nota":408.5454587}
    {"_id":4,"pais":"Albania","continente":"Europe","materia":"REA","nota":405.2588}
    
La materia puede ser MAT (matemáticas) REA (lectura) o SCI (ciencias). Además cada materia puede tener dos variantes: _FE para mujeres y _MA para hombres.     

## Inicializar Mongo

Primero las librerías:

In [1]:
# pymongo
try:
    import pymongo
    print("Pymongo está en el sistema!")
except ImportError as e:
    !{sys.executable} -m pip install --upgrade --user pymongo
    
# pprint
try:
    import pprint
    from pprint import pprint
    print("pprint está en el sistema!")
except ImportError as e:
    !{sys.executable} -m pip install --upgrade --user pprint
   


Pymongo está en el sistema!
pprint está en el sistema!


In [2]:
from pymongo import MongoClient
# local
conexion = 'mongodb://127.0.0.1:27017/'

client = MongoClient(conexion)

# código para ver si se ha conectado bien
try:
    s = client.server_info() # si hay error tendremos una excepción
    print("Conectado a MongoDB, versión",s["version"])
    db = client.educacion
    colecciones = db.list_collection_names()
    for col in colecciones:
        print('Colección ',col,'. Total documentos: ',db[col].count_documents({})) 
except:
    print ("connection error")

Conectado a MongoDB, versión 4.4.1
Colección  pisa . Total documentos:  567
Colección  continenMat . Total documentos:  5


# Preguntas

Utilizar aggregate para resolver las siguientes preguntas. 

Se tendrá en cuenta la eficiencia (no hacer operaciones innecesarias y hacerlas en el orden adecuado)

Recordar que estamos en Python y hay que poner comillas en cada clave: `{"$group":{"_id":...}}`

### Pregunta 1 (1 punto)
Queremos saber la media de la clave 'nota' para cada clave 'pais'. El resultado debe estar por orden de media, de mayor a menor (para comprobar, el primero debe ser 'Singapore')

In [3]:
# cambiar el pipeline [] por el que proceda
pipe = [
    {"$group":
        {
            "_id":"$pais",
            "media": {"$avg":'$nota'}
        }
    }
    ,{"$sort":{'media':-1}}
]
res = db.pisa.aggregate(pipe)
for doc in res:
    pprint(doc)

{'_id': 'Singapore', 'media': 551.6731830666666}
{'_id': 'Japan', 'media': 528.9177938333333}
{'_id': 'Estonia', 'media': 524.3320617666667}
{'_id': 'Canada', 'media': 523.3373938444445}
{'_id': 'Finland', 'media': 523.0064316222222}
{'_id': 'Slovenia', 'media': 509.49883024444443}
{'_id': 'Ireland', 'media': 508.9948420888889}
{'_id': 'Germany', 'media': 508.05964173333336}
{'_id': 'Netherlands', 'media': 507.92265712222223}
{'_id': 'Switzerland', 'media': 506.3512308555555}
{'_id': 'New Zealand', 'media': 505.94557935555554}
{'_id': 'Norway', 'media': 504.52313141111114}
{'_id': 'Denmark', 'media': 504.28322165555556}
{'_id': 'Poland', 'media': 503.8906867555556}
{'_id': 'Belgium', 'media': 502.4849992888889}
{'_id': 'Australia', 'media': 502.28441944444444}
{'_id': 'Vietnam', 'media': 501.89859955555556}
{'_id': 'United Kingdom', 'media': 499.90720028888893}
{'_id': 'Portugal', 'media': 496.94857237777774}
{'_id': 'Sweden', 'media': 495.87855532222216}
{'_id': 'France', 'media': 495

### Pregunta 2 (1 punto)

Lo mismo pero limitado al país 'Spain' (en este caso no hace falta ordenar, claro, es un solo documento)

In [4]:
# cambiar el pipeline [] por el que proceda
pipe = [ 
    {"$match":{"pais":"Spain"}},
    {"$group":
        {
            "_id":"$pais",
            "media": {"$avg":'$nota'}
        }
    }
]
res = db.pisa.aggregate(pipe)
for doc in res:
    pprint(doc)

{'_id': 'Spain', 'media': 491.4022776777778}


### Pregunta 3 (1 punto)
Ahora queremos repetir la consulta 1, pero de forma que la consulta devuelva solo aquellos documentos correspondientes a países con media igual o superior a 500. Mostrarlos por orden de media, de menor a mayor

In [5]:
pipe = [    
    {"$group":
        {
            "_id":"$pais",
            "media": {"$avg":'$nota'}
        }
    }
    ,{"$match":{'media':{"$gte":500}}}
    ,{"$sort":{'media':1}}
]
res = db.pisa.aggregate(pipe)
for doc in res:
    pprint(doc)

{'_id': 'Vietnam', 'media': 501.89859955555556}
{'_id': 'Australia', 'media': 502.28441944444444}
{'_id': 'Belgium', 'media': 502.4849992888889}
{'_id': 'Poland', 'media': 503.8906867555556}
{'_id': 'Denmark', 'media': 504.28322165555556}
{'_id': 'Norway', 'media': 504.52313141111114}
{'_id': 'New Zealand', 'media': 505.94557935555554}
{'_id': 'Switzerland', 'media': 506.3512308555555}
{'_id': 'Netherlands', 'media': 507.92265712222223}
{'_id': 'Germany', 'media': 508.05964173333336}
{'_id': 'Ireland', 'media': 508.9948420888889}
{'_id': 'Slovenia', 'media': 509.49883024444443}
{'_id': 'Finland', 'media': 523.0064316222222}
{'_id': 'Canada', 'media': 523.3373938444445}
{'_id': 'Estonia', 'media': 524.3320617666667}
{'_id': 'Japan', 'media': 528.9177938333333}
{'_id': 'Singapore', 'media': 551.6731830666666}


### Pregunta 4 (1 punto)
Mostrar solo los documentos con materia SCI (sin agrupar por nada, solo estos documentos), pero quitando el continente y el _id. Mostrar por orden de nombre de país. La salida será de la forma:

    {'materia': 'SCI', 'nota': 427.225, 'pais': 'Albania'}
    {'materia': 'SCI', 'nota': 375.7451, 'pais': 'Algeria'}

In [6]:
pipe = [
    {"$match":{"materia":"SCI"}},
    {"$project":{
        "_id":0,
        "continente":0,
    }},
    {"$sort":{"pais":1}}
    
]
res = db.pisa.aggregate(pipe)
for doc in res:
    pprint(doc)

{'materia': 'SCI', 'nota': 427.225, 'pais': 'Albania'}
{'materia': 'SCI', 'nota': 375.7451, 'pais': 'Algeria'}
{'materia': 'SCI', 'nota': 432.2262, 'pais': 'Argentina'}
{'materia': 'SCI', 'nota': 509.9939, 'pais': 'Australia'}
{'materia': 'SCI', 'nota': 495.0375, 'pais': 'Austria'}
{'materia': 'SCI', 'nota': 501.9997, 'pais': 'Belgium'}
{'materia': 'SCI', 'nota': 400.6821, 'pais': 'Brazil'}
{'materia': 'SCI', 'nota': 445.772, 'pais': 'Bulgaria'}
{'materia': 'SCI', 'nota': 527.7047, 'pais': 'Canada'}
{'materia': 'SCI', 'nota': 446.9561, 'pais': 'Chile'}
{'materia': 'SCI', 'nota': 415.7288, 'pais': 'Colombia'}
{'materia': 'SCI', 'nota': 419.608, 'pais': 'Costa Rica'}
{'materia': 'SCI', 'nota': 475.3912, 'pais': 'Croatia'}
{'materia': 'SCI', 'nota': 432.5964, 'pais': 'Cyprus'}
{'materia': 'SCI', 'nota': 492.83, 'pais': 'Czech Republic'}
{'materia': 'SCI', 'nota': 501.9369, 'pais': 'Denmark'}
{'materia': 'SCI', 'nota': 331.6388, 'pais': 'Dominican Republic'}
{'materia': 'SCI', 'nota': 534.

### Pregunta 5 (1 punto) 

Ahora queremos hacer de nuevo primero la media por países, como en la consulta 1, pero después hacer la media de esas medias por continentes. Mostrar el resultado ordenado alfabéticamente por orden de nombre de continente


In [7]:
pipe = [    
    {"$group":
        {
            "_id":{"pais":"$pais", "continente":"$continente"},
            "mediaPais": {"$avg":'$nota'}
        }
    }
    ,{"$group":
         {
             "_id":"$_id.continente",
             "media":{"$avg":"$mediaPais"}
         }
     }
    ,{"$sort":{'_id':1}}
    
]
res = db.pisa.aggregate(pipe)
for doc in res:
    pprint(doc)

{'_id': 'Africa', 'media': 366.6750802}
{'_id': 'America', 'media': 424.83272813240745}
{'_id': 'Asia', 'media': 451.6022277838384}
{'_id': 'Europe', 'media': 480.30074316574076}
{'_id': 'Oceania', 'media': 504.1149994}


### Pregunta 6 [1.5]

Nos interesa tener un solo documento por cada país, incluyendo todas sus notas

    {'_id': 'Albania',
     'notas': [{'materia': 'SCI', 'nota': 427.225},
               {'materia': 'REA', 'nota': 405.2588},
               {'materia': 'MAT_FE', 'nota': 417.7500295},
               {'materia': 'MAT', 'nota': 413.157},
               {'materia': 'SCI_FE', 'nota': 439.4429629},
               {'materia': 'MAT_MA', 'nota': 408.5454587},
               {'materia': 'REA_FE', 'nota': 434.63962549999997},
               {'materia': 'REA_MA', 'nota': 375.7591992},
               {'materia': 'SCI_MA', 'nota': 414.9576437}]}

Escribir una consulta para obtener documentos de este tipo

In [8]:
pipe = [
    {"$group":
        
         {"_id":"$pais",
         "notas":{"$push":{"materia":"$materia","nota":"$nota"}}
         }
    }
    ,{"$sort":{"_id":1}}
    
]
res = db.pisa.aggregate(pipe)
for doc in res:
    pprint(doc)

{'_id': 'Albania',
 'notas': [{'materia': 'REA', 'nota': 405.2588},
           {'materia': 'REA_FE', 'nota': 434.63962549999997},
           {'materia': 'REA_MA', 'nota': 375.7591992},
           {'materia': 'MAT', 'nota': 413.157},
           {'materia': 'MAT_FE', 'nota': 417.7500295},
           {'materia': 'MAT_MA', 'nota': 408.5454587},
           {'materia': 'SCI_MA', 'nota': 414.9576437},
           {'materia': 'SCI', 'nota': 427.225},
           {'materia': 'SCI_FE', 'nota': 439.4429629}]}
{'_id': 'Algeria',
 'notas': [{'materia': 'MAT', 'nota': 359.6062},
           {'materia': 'MAT_FE', 'nota': 363.0724791},
           {'materia': 'MAT_MA', 'nota': 356.4951057},
           {'materia': 'REA', 'nota': 349.8593},
           {'materia': 'REA_FE', 'nota': 366.2081668},
           {'materia': 'SCI_FE', 'nota': 383.2209389},
           {'materia': 'SCI_MA', 'nota': 369.0352338},
           {'materia': 'SCI', 'nota': 375.7451},
           {'materia': 'REA_MA', 'nota': 335.185435899999

### Pregunta 7 [1.5]
Para la materia 'MAT', queremos mostrar cuántos países tienen nota entre 0 y 400 , 400 y 450, 450 y 500 y quinientos o más.  Nota: fijad los límites (intervalos cerrados o abiertos) como queráis 


In [9]:
pipe = [
    {"$match":{"materia":"MAT"}},
    {"$bucket":{
        "groupBy":"$nota",
         "boundaries":[0,400,450,500],
         "default":"Los mejores",
         "output":{
             "cuantos":{"$sum":1}
         }
    }
    }
]
res = db.pisa.aggregate(pipe)
for doc in res:
    pprint(doc)

{'_id': 0, 'cuantos': 8}
{'_id': 400, 'cuantos': 17}
{'_id': 450, 'cuantos': 24}
{'_id': 'Los mejores', 'cuantos': 14}


### Pregunta 8 [2 puntos]

Queremos tener un documento por continente que contenga:
 - Como _id  el nombre de ese continente
 - Una clave `mediaMatMal` con la  media de notas "MAT" de los países de ese continente que tengan nota de "MAT" menor de 500 (se puede asumir que hay al menos 1)
 - Una clave `numMatMal` con el número de países del continente que cumplen la condición anterior
 - un array  `paises`  con los documentos de todos los paises de ese continente. 
 
 El resultado debe quedar grabado en una colección de nombre `continenMat`

In [11]:
pipe = [
    {"$match":{"materia":"MAT", 'nota':{"$lt":500}}}
    ,{"$group":
         {
             "_id":"$continente",
             "numMatMal":{"$sum":1},
             "mediaMatMal":{"$avg": "$nota"},
             "paises":{"$push":{"_id":"$_id","pais":"$pais","continente":"$continente","materia":"$materia","nota":"$nota"}}
         }
     }
    ,{"$out": "continenMat"}
]
res = db.pisa.aggregate(pipe)
for doc in res:
    pprint(doc)
   